In [3]:
import os
import cv2
from roboflow import Roboflow
from google.colab.patches import cv2_imshow

In [4]:
rf = Roboflow(api_key=" ")
project = rf.workspace("utkarsh-srivastava-3n6fv").project("projectcv-kc4ww")
model_yolo = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [5]:
# Load the video
video_path = "Guns_video.mp4"
cap = cv2.VideoCapture(video_path)

In [8]:
# Define the codec and create a VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to image for prediction
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Make prediction
    predictions = model_yolo.predict(frame_rgb)

    # Draw bounding boxes on the frame
    for prediction in predictions:
        x, y, w, h = prediction['x'], prediction['y'], prediction['width'], prediction['height']
        label = prediction['class']
        confidence = prediction['confidence']

        # Calculate bounding box coordinates
        start_point = (int(x - w / 2), int(y - h / 2))
        end_point = (int(x + w / 2), int(y + h / 2))

        # Draw rectangle
        cv2.rectangle(frame, start_point, end_point, (0, 255, 0), 2)
        cv2.putText(frame, f"{label} ({confidence:.2f})", (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Output in the mp4 format
    out.write(frame)

    # Break the loop if 'q' is pressed (optional)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and writer objects
cap.release()
out.release()

In [ ]:
# Download the video to your local machine (for colab only)
from google.colab import files
files.download('output.mp4')